In [ ]:
import torch
from PIL import Image
import os
import pandas as pd
from torch.utils.data import Dataset
import numpy as np

class ShapeImageDataset(Dataset):
    def __init__(self, df, images, transform=None):
        self.df = pd.read_csv(df)
        self.images = images
        self.transform = transform
        self.emotion_columns = self.df.columns[7:] 
        
    def __len__(self):
        image_files = os.listdir(self.images)
        dataset_size = min(len(self.df), len(image_files)) 
        return dataset_size
    
    def __getitem__(self, idx):
        adjusted_idx = idx + 1
        
        if adjusted_idx > len(self.df) or adjusted_idx > len(os.listdir(self.images)):
            return None, None  
        
        row = self.df.iloc[adjusted_idx - 1]  
        image_path = os.path.join(self.images, f"{adjusted_idx}.png")  

        try:
            image = Image.open(image_path).convert("RGB")  
        except Exception:
            return None, None  

        if self.transform:
            image = self.transform(image)

        labels = row[self.emotion_columns].fillna(0).values.astype(np.float32)  

        label_tensor = torch.tensor(labels, dtype=torch.float32)

        return image, label_tensor


Loaded CSV file with 80 rows and 28 emotion columns.
Emotion columns being checked: ['Admiration', 'Amusement', 'Anger', 'Annoyance', 'Approval', 'Caring', 'Confusion', 'Curiosity', 'Desire', 'Disappointment', 'Disapproval', 'Disgust', 'Embarassment', 'Excitement', 'Fear', 'Gratitude', 'Grief', 'Joy', 'Love', 'Nervousness', 'Optimism', 'Pride', 'Realization', 'Relief', 'Remorse', 'Sadness', 'Suprise', 'Neutral']
Testing index 1...
Getting item at adjusted index 1...
Loading image for index 1: data/images\1.png
Successfully loaded image data/images\1.png with size: (200, 200)
Extracted labels for index 1: [ 0.  0.  0. 20.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 40.  0.  0.  0.
  0. 20.  0.  0.  0.  0. 20.  0.  0.  0.]
Checking columns for labels: ['Admiration', 'Amusement', 'Anger', 'Annoyance', 'Approval', 'Caring', 'Confusion', 'Curiosity', 'Desire', 'Disappointment', 'Disapproval', 'Disgust', 'Embarassment', 'Excitement', 'Fear', 'Gratitude', 'Grief', 'Joy', 'Love', 'Nervousness', 'O

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.models import resnet50

# Image transformations
transform = transforms.Compose([
    transforms.Resize((200, 200)),  # Resize images to (200, 200)
    transforms.ToTensor(),          # Convert images to tensor format
])

# Dataset and DataLoader
dataset = ShapeImageDataset(df='data/shapes.csv', images='data/images', transform=transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# Load pre-trained ResNet50 and modify for 28 emotions
model = resnet50(pretrained=True)
num_emotions = 28  # Since you have 28 emotions as output
model.fc = nn.Linear(model.fc.in_features, num_emotions)  # Change the final layer for 28 outputs

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Loss and optimizer
criterion = nn.BCEWithLogitsLoss()  # Use BCEWithLogitsLoss for multi-label classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 10
for epoch in range(epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0
    
    for batch_idx, (images, labels) in enumerate(dataloader):
        # Skip any iteration where images or labels are None
        if images is None or labels is None:
            continue

        # Move images and labels to the device (GPU or CPU)
        images = images.to(device)
        labels = labels.to(device)  # Now labels are tensors

        # Forward pass
        outputs = model(images)

        # Calculate loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track the running loss
        running_loss += loss.item()

    # Print average loss for this epoch
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss / len(dataloader):.4f}")

torch.save(model.state_dict(), 'resnet50_shapes.pth')




Loaded CSV file with 80 rows and 28 emotion columns.
Emotion columns being checked: ['Admiration', 'Amusement', 'Anger', 'Annoyance', 'Approval', 'Caring', 'Confusion', 'Curiosity', 'Desire', 'Disappointment', 'Disapproval', 'Disgust', 'Embarassment', 'Excitement', 'Fear', 'Gratitude', 'Grief', 'Joy', 'Love', 'Nervousness', 'Optimism', 'Pride', 'Realization', 'Relief', 'Remorse', 'Sadness', 'Suprise', 'Neutral']
Dataset size: 80 (from 80 rows and 80 images)
Dataset size: 80 (from 80 rows and 80 images)


c:\Users\TTill\Anaconda3\envs\keji\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\TTill\Anaconda3\envs\keji\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Dataset size: 80 (from 80 rows and 80 images)
Dataset size: 80 (from 80 rows and 80 images)
Getting item at adjusted index 70...
Loading image for index 70: data/images\70.png
Successfully loaded image data/images\70.png with size: (200, 200)
Applied transformations to the image. New size: torch.Size([3, 200, 200])
Extracted labels for index 70: [ 0.        0.        0.        0.       14.285714 14.285714  0.
  0.        0.        0.        0.        0.       14.285714 14.285714
 14.285714 14.285714  0.        0.        0.       14.285714  0.
  0.        0.        0.        0.        0.        0.        0.      ]
Checking columns for labels: ['Admiration', 'Amusement', 'Anger', 'Annoyance', 'Approval', 'Caring', 'Confusion', 'Curiosity', 'Desire', 'Disappointment', 'Disapproval', 'Disgust', 'Embarassment', 'Excitement', 'Fear', 'Gratitude', 'Grief', 'Joy', 'Love', 'Nervousness', 'Optimism', 'Pride', 'Realization', 'Relief', 'Remorse', 'Sadness', 'Suprise', 'Neutral']
Converted labels t